# Author Bot

A helper to support you when writing a book!

You will need an OpenAI API key, which you can get from [OpenAI.com](https://openai.com). Create a `.env` file by renaming the existing `.env.example` to `.env`, then set the value of `OPENAI_KEY` to your OpenAI key.

In [ ]:
# Load the OpenAI key from the .env file
%load_ext dotenv
%dotenv

import os
OPENAI_KEY = os.environ['KEY']

Set the screen width when showing the text - if the text wraps in the output window, make this number smaller.

In [ ]:
SCREEN_WIDTH=120

Create the OpenAI chat completion object.

In [ ]:
import openai

# Load your API key from an environment variable or secret management service
openai.api_key = OPENAI_KEY

# If you are using Azure OpenAI service, uncomment the next few lines fill in the details
# openai.api_base = os.environ['ENDPOINT']
# openai.api_type = 'azure'
# openai.api_version = '2023-05-15'

# list models
models = openai.Model.list()
for model in (m for m in models['data'] if 'gpt' in m['id']):
    print(model['id'])

# Select the GPT-4 model
MODEL = 'gpt-4'

Create the system prompt. The system prompt provides basic guidance for the chat completions API to use for all requests. The system prompt is sent to the API as a message of type `system`.

In this case the system prompt defines the genre and the audience for the book. This system prompt is added to a list of messages.

In [ ]:
genre = 'science fiction'
# genre = 'fantasy'
# genre = 'horror'
# genre = 'romance'
# genre = 'mystery'
# genre = 'thriller'
# genre = 'western'
# genre = 'drama'
# genre = 'comedy'
# genre = 'action'
# genre = 'adventure'
# genre = 'historical fiction'
# genre = 'literary fiction'

audience = 'adults'
# audience = 'teenagers'
# audience = 'children'

system_prompt_message = {
    'role': 'system',
    'content': f'You are a bot designed to help an author write a best selling book. The author is writing a {genre} book for {audience}. This book should be cheesy and pulpy to target the widest possible audience'
}

Now we have the system prompt, we can use a chat completion to get a selection of possible plots for the book.

In [ ]:
import json

plot_request_message = {
    'role': 'user',
    'content': 'Provide 3 possible plots for the book. Output the plots along with between 3 and 5 main characters as JSON in the following format: [{ "plot" : "plot", "characters" : [{"name", "character full name", "occupation", "character occupation"}]}]. Do not start the plot with the plot number.'
}

messages = [
    system_prompt_message,
    plot_request_message
]

chat_completion = openai.ChatCompletion.create(model=MODEL, messages=messages)

# Sometimes the response has the JSON with the json keys with the first letter capitalized, so normalize on lowercase
def normalize_json_keys(json_dict):
    result = {}
    for key in json_dict.keys():
        result[key.lower()] = json_dict[key]
    return result

def load_openai_json(json_string):
    loaded = json.loads(json_string)

    # if loaded is a list, then it is a list of JSON strings, so load each one
    if isinstance(loaded, list):
        return [normalize_json_keys(item) for item in loaded]
    
    return normalize_json_keys(loaded)
    

book_ideas = load_openai_json(chat_completion.choices[0].message.content)

The `chat_completion` returns the result containing the response as an `assistant` message, in the JSON format we specified.

In [ ]:

import textwrap

def print_book_idea(book_idea):
    print('Plot:')
    print('')
    print(textwrap.fill(book_idea['plot'], width=SCREEN_WIDTH, replace_whitespace=False))
    print('')
    print('Characters:')
    print('')
    for character in book_idea['characters']:
        print(f'{character["name"]} - {character["occupation"]}')
    print('')

for book_idea in book_ideas:
    print_book_idea(book_idea)

Now we have our ideas, let's select one and run with it. In the next cell, set the index from 0-2 of the book idea you want to work on into the `SELECTED_BOOK_IDEA` constant.

In [ ]:
# Select your book idea by setting this index
SELECTED_BOOK_IDEA = 2

book_idea = book_ideas[SELECTED_BOOK_IDEA]

print_book_idea(book_idea)

Now we have the plot and the characters, we can put these into the system message so any follow up completions are based on this.

In [ ]:
book_plot = book_idea['plot']

book_details_message = { 'role': 'assistant', 'content': f'The book has the following plot: "{book_plot}".' }

book_characters = 'The book has the following characters: '
for character in book_idea['characters']:
    book_characters += f'{character["name"]} who is a {character["occupation"]}, '

book_characters = book_characters[:-2] + '.'

book_characters_message = { 'role': 'assistant', 'content': book_characters }

print(textwrap.fill(book_details_message['content'], width=SCREEN_WIDTH, replace_whitespace=False))
print(textwrap.fill(book_characters_message['content'], width=SCREEN_WIDTH, replace_whitespace=False))

Now we can expand on the plot by asking for a longer plot.

In [ ]:
messages = [
    system_prompt_message,
    book_details_message,
    book_characters_message,
    { 'role': 'user', 'content': 'Come up with a catchy title for this book, then write a 1000 character plot summary of the book. Make one of the characters a villain. Output as JSON in the following format" { "title": "title", "plot_summary" : "plot summary", "villain", "Name of the villain"}'}
]

chat_completion = openai.ChatCompletion.create(model=MODEL, messages=messages)

# Extract the title, plot summary and villain from the response
response = load_openai_json(chat_completion.choices[0].message.content)
book_title = response['title']
book_plot_summary = response['plot_summary']
book_villain = response['villain']

print(f'Title: {book_title}')
print('')
print('Plot Summary:')
print('')
print(textwrap.fill(book_plot_summary, width=SCREEN_WIDTH, replace_whitespace=False))
print('')
print(f'Villain: {book_villain}')

We can now use this plot to ask for detailed character biographies, along with any details of their relationships to each other. We start by adding the output as an assistant message.

In [ ]:
# Rebuild the book details message with the new plot summary
book_details_message = { 'role': 'assistant', 'content': f'The book is called {book_title} has the following plot: "{book_plot_summary}".' }
book_characters_message = { 'role': 'assistant', 'content': f'{book_characters}. {book_villain} is the villain in this story.' }

messages = [
    system_prompt_message,
    book_details_message,
    book_characters_message,
    { 'role': 'user', 'content': 'Write a 500 character biography for each of the character, including if they have any relationships to each other. Also write a 500 character visual description. Output as JSON in the following format" { "name": "name", "occupation" : "occupation", "biography", "biography", "description": "visual description"}'}
]

# Generate the character biographies
chat_completion = openai.ChatCompletion.create(model=MODEL, messages=messages)
print(textwrap.fill(chat_completion.choices[0].message.content, width=SCREEN_WIDTH, replace_whitespace=False))

# Capture the response
character_response = load_openai_json(chat_completion.choices[0].message.content)

Now we can start creating the outline of the book using all the details we have.

In [ ]:
# Rebuild the character details message with the new details
book_characters = f'The book has the following characters:\n'

character_count = 1

for character in character_response:
    if character['name'] == book_villain:
        book_characters += f'{character_count}. {character["name"]} who is a {character["occupation"]}, and is the villain in this story. {character["biography"]} {character["description"]}\n'
    else:
        book_characters += f'{character_count}. {character["name"]} who is a {character["occupation"]}. {character["biography"]} {character["description"]}\n'
    character_count += 1

book_characters_message = { 'role': 'assistant', 'content': f'{book_characters[:-1]}' }

messages = [
    system_prompt_message,
    book_details_message,
    book_characters_message,
    { 'role': 'user', 'content': 'Create 20 chapters for the book, with a chapter name and a 200 character detailed summary for each chapter. Return a JSON array in the following format" { "chapter": "chapter", "chapter_name" : "chapter name", "summary" : "summary"}. Just return JSON and no prose'}
]

chat_completion = openai.ChatCompletion.create(model=MODEL, messages=messages)
print(textwrap.fill(chat_completion.choices[0].message.content, width=SCREEN_WIDTH, replace_whitespace=False))

# Capture the response
chapter_response = load_openai_json(chat_completion.choices[0].message.content)

Build the chapters into the prompt

In [ ]:
book_chapters = f'The book has the following chapters:\n'

for chapter in chapter_response:
    book_chapters += f'{chapter["chapter"]}, {chapter["chapter_name"]}. {chapter["summary"]}\n'

book_chapters_message = { 'role': 'assistant', 'content': f'{book_chapters[:-1]}' }
print(book_chapters)

We have a lot of data! ChatGPT has limits on tokens, so we need to see how many tokens we have so far.

In [ ]:
import tiktoken

def num_tokens_from_messages(messages, model):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

def max_tokens(model):
    if 'gpt-3' in model:
        if '16K' in model:
            return 16384
        else:
            return 4096
    elif 'gpt-4' in model:
        if '32K' in model:
            return 32768
        else:
            return 8192
    else:
        return -1

messages = [
    system_prompt_message,
    book_details_message,
    book_characters_message,
    book_chapters_message
]

tokens_used = num_tokens_from_messages(messages, model=MODEL)
max_tokens = max_tokens(MODEL)
print(f'Tokens used: {tokens_used}, max tokens: {max_tokens}, remaining: {max_tokens - tokens_used}')

As long as this is below the limit, we are fine. The limit for GPT-3 is 4K, GPT-4 is 8K. This includes the tokens sent and received, hence why the maximum we can get per call is the maximum for the model minus what we send.
We can now create the chapters. Let's loop through them and write the chapters out to different files

In [ ]:
# Create a directory for the book in the books directory

# Create the books directory if it doesn't exist
if not os.path.exists('./books'):
    os.makedirs('./books')

# Create a file for the book
book_filename = f'./books/{book_title.replace(" ", "_").replace(":", "")}.md'

if os.path.exists(book_filename):
    os.remove(book_filename)

file = open(book_filename, 'w')

file.write(f'# {book_title}')

chapter_text = None

# For each chapter, create a file with the chapter number and the chapter title
chapter_num = 1
for chapter in chapter_response:
    file.write(f'\n\n## Chapter {chapter_num} - {chapter["chapter_name"]}\n\n')

    if chapter_text is not None:
        chapter_message = { 'role': 'user', 'content': f"Write chapter {chapter_num}. Don't include the chapter number or the chapter name. The previous chapter is ```{chapter_text}```"}
    else:
        chapter_message = { 'role': 'user', 'content': f"Write chapter {chapter_num}. Don't include the chapter number or the chapter name."}

    messages = [
        system_prompt_message,
        book_details_message,
        book_characters_message,
        book_chapters_message,
        chapter_message
    ]
    
    chat_completion = openai.ChatCompletion.create(model=MODEL, messages=messages)
    chapter_text = chat_completion.choices[0].message.content

    file.write(chapter_text)
    print(f'Wrote chapter {chapter_num} to {chapter["chapter_name"]}')

    file.flush()

    chapter_num += 1